In [2]:
!pip install cryptography
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.backends import default_backend
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import padding, rsa
from cryptography.exceptions import InvalidSignature
from pathlib import Path
import os
from cryptography.hazmat.backends import default_backend
import requests
import datetime

In [3]:
# Cell 1 — load KALSHI keys from .env in the same directory as this notebook


# --- Try to use python-dotenv if it's installed (recommended) ---
try:
    from dotenv import load_dotenv  # pip install python-dotenv
    env_path = Path.cwd() / ".env"   # assumes notebook is run from the same directory as .env
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")
    load_dotenv(dotenv_path=env_path, override=False)
except ImportError:
    # --- Fallback: minimal .env parser (no extra install needed) ---
    env_path = Path.cwd() / ".env"
    if not env_path.exists():
        raise FileNotFoundError(f".env not found at: {env_path}")

    for line in env_path.read_text().splitlines():
        line = line.strip()
        if not line or line.startswith("#") or "=" not in line:
            continue
        k, v = line.split("=", 1)
        k = k.strip()
        v = v.strip().strip('"').strip("'")
        os.environ.setdefault(k, v)  # don't overwrite if already set

# --- Read your keys into variables ---
public_key = os.getenv("KALSHI-ACCESS-KEY-DEMO")

missing = [k for k, v in {
    "KALSHI-ACCESS-KEY": public_key
}.items() if not v]

if missing:
    raise KeyError(f"Missing env var(s): {missing}. Check your .env formatting and key names.")

print("Loaded Kalshi keys ✅")  # intentionally not printing the secret values


def load_private_key_from_file(file_path):
    with open(file_path, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None,  # or provide a password if your key is encrypted
            backend=default_backend()
        )
    return private_key


def sign_pss_text(private_key: rsa.RSAPrivateKey, text: str) -> str:
    message = text.encode('utf-8')
    try:
        signature = private_key.sign(
            message,
            padding.PSS(
                mgf=padding.MGF1(hashes.SHA256()),
                salt_length=padding.PSS.DIGEST_LENGTH
            ),
            hashes.SHA256()
        )
        return base64.b64encode(signature).decode('utf-8')
    except InvalidSignature as e:
        raise ValueError("RSA sign PSS failed") from e
    

def make_authenticated_request(path, method='GET', params=None):
    """Make an authenticated request to Kalshi demo API."""
    current_time = datetime.datetime.now()
    timestamp = current_time.timestamp()
    current_time_milliseconds = int(timestamp * 1000)
    timestamp_str = str(current_time_milliseconds)
    
    private_key = load_private_key_from_file('pbhaskarademo.txt')
    
    base_url = 'https://demo-api.kalshi.co'
    
    # Strip query parameters from path before signing
    path_without_query = path.split('?')[0]
    msg_string = timestamp_str + method + path_without_query
    sig = sign_pss_text(private_key, msg_string)
    
    headers = {
        'KALSHI-ACCESS-KEY': public_key,
        'KALSHI-ACCESS-SIGNATURE': sig,
        'KALSHI-ACCESS-TIMESTAMP': timestamp_str
    }
    
    if method == 'GET':
        response = requests.get(base_url + path, headers=headers, params=params)
    else:
        response = requests.post(base_url + path, headers=headers, json=params)
    
    return response

Loaded Kalshi keys ✅


In [4]:
# ============================================================================
# STEP 1: Pull All Historical Companies Markets
# ============================================================================

import requests
import time
from datetime import datetime, timedelta, timezone
import numpy as np
import pandas as pd
try:
    from zoneinfo import ZoneInfo
    ET_TZ = ZoneInfo('America/New_York')
except ImportError:
    ET_TZ = timezone(timedelta(hours=-5))

print("="*80)
print("FETCHING HISTORICAL COMPANIES MARKETS")
print("="*80)

# Calculate time window (past year)
now_comp = datetime.now(timezone.utc)
one_year_ago_comp = now_comp - timedelta(days=365)
min_close_ts_comp = int(one_year_ago_comp.timestamp())
max_close_ts_comp = int(now_comp.timestamp())

print(f"\nTime window:")
print(f"  From: {one_year_ago_comp.strftime('%Y-%m-%d %H:%M:%S UTC')}")
print(f"  To:   {now_comp.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# Search for Companies category series
print("\n" + "-"*80)
print("Fetching Companies category series...")
print("-"*80)

base_url_comp = "https://api.elections.kalshi.com/trade-api/v2"

# Fetch series by category "Companies"
companies_series = []

try:
    response = requests.get(f"{base_url_comp}/series", params={"category": "Companies", "limit": 200})
    if response.status_code == 200:
        series_data = response.json().get('series', [])
        companies_series.extend(series_data)
        print(f"  ✓ Found {len(series_data)} series in Companies category")
    else:
        print(f"  ✗ Error: Status {response.status_code}")
    time.sleep(0.5)
except Exception as e:
    print(f"  ✗ Error fetching Companies category: {e}")

# If category search fails, try keyword-based search
if len(companies_series) == 0:
    print("\n  Category search returned no results. Trying keyword search...")
    
    # Try different company-related keywords
    search_keywords = ["stock", "company", "earnings", "revenue", "nasdaq", "dow", "sp500"]
    
    for keyword in search_keywords:
        try:
            response = requests.get(f"{base_url_comp}/series", params={"limit": 200})
            if response.status_code == 200:
                all_series = response.json().get('series', [])
                # Filter by keyword
                keyword_series = [
                    s for s in all_series 
                    if keyword.lower() in s.get('title', '').lower() 
                    or keyword.lower() in s.get('category', '').lower()
                    or any(keyword.lower() in tag.lower() for tag in s.get('tags', []))
                ]
                companies_series.extend(keyword_series)
                if keyword_series:
                    print(f"    Found {len(keyword_series)} series matching '{keyword}'")
            time.sleep(0.5)
        except Exception as e:
            print(f"    Error searching for '{keyword}': {e}")

# Deduplicate series
unique_companies_series = {s['ticker']: s for s in companies_series}.values()
print(f"\n✓ Total unique companies series: {len(unique_companies_series)}")

# Display found series
if unique_companies_series:
    print("\nCompanies series found:")
    for s in list(unique_companies_series)[:10]:  # Show first 10
        print(f"  - {s['ticker']}: {s['title']}")
        print(f"    Category: {s.get('category', 'N/A')}")
    if len(unique_companies_series) > 10:
        print(f"  ... and {len(unique_companies_series) - 10} more series")

# Store series tickers
companies_series_tickers = [s['ticker'] for s in unique_companies_series]
companies_series_list = list(unique_companies_series)

print(f"\n✓ Will fetch markets for {len(companies_series_tickers)} companies series")
print(f"{'='*80}")

FETCHING HISTORICAL COMPANIES MARKETS

Time window:
  From: 2025-02-01 08:26:59 UTC
  To:   2026-02-01 08:26:59 UTC

--------------------------------------------------------------------------------
Fetching Companies category series...
--------------------------------------------------------------------------------
  ✓ Found 349 series in Companies category

✓ Total unique companies series: 349

Companies series found:
  - KXDANGROSSMETA: Daniel Gross joining Meta
    Category: Companies
  - KXTESLAPROD: Tesla production
    Category: Companies
  - CBVOLUME: Coinbase volume
    Category: Companies
  - KXTIKTOKSELL: Congress passes law to force TikTok sale
    Category: Companies
  - KXTOPLLM: GPT no longer top ranked LLM
    Category: Science and Technology
  - KXANTITRUSTOAIMSFT: OpenAI Microsoft antitrust
    Category: Companies
  - KXGROKMACOS: Grok MacOS app
    Category: Companies
  - KXNEWPRODUCTFALLOUT3REM: “will Fallout 3 remastered be revealed on October 23rd?”
    Category: C

In [5]:
# ============================================================================
# STEP 2: Fetch All Markets for Companies Series
# ============================================================================

print("\n" + "="*80)
print("FETCHING MARKETS FOR COMPANIES SERIES")
print("="*80)

# Fetch all markets for companies series
all_companies_markets = []

print(f"\nFetching settled markets for {len(companies_series_tickers)} series...")
print("(This may take a few minutes...)\n")

for i, series_ticker in enumerate(companies_series_tickers, 1):
    cursor = None
    series_markets = []
    
    while True:
        params = {
            "series_ticker": series_ticker,
            "status": "settled",
            "min_close_ts": min_close_ts_comp,
            "max_close_ts": max_close_ts_comp,
            "limit": 1000
        }
        
        if cursor:
            params["cursor"] = cursor
        
        try:
            response = requests.get(f"{base_url_comp}/markets", params=params)
            
            if response.status_code == 429:
                print(f"  Rate limited, waiting 2 seconds...")
                time.sleep(2)
                continue
            
            if response.status_code != 200:
                print(f"  ✗ Error for {series_ticker}: Status {response.status_code}")
                break
            
            data = response.json()
            markets = data.get('markets', [])
            series_markets.extend(markets)
            
            cursor = data.get('cursor')
            if not cursor:
                break
            
            time.sleep(0.3)
            
        except Exception as e:
            print(f"  ✗ Error fetching {series_ticker}: {e}")
            break
    
    if series_markets:
        all_companies_markets.extend(series_markets)
        print(f"  [{i}/{len(companies_series_tickers)}] {series_ticker}: {len(series_markets)} markets")
    
    if i % 5 == 0:
        print(f"  Progress: {i}/{len(companies_series_tickers)} series processed, {len(all_companies_markets)} total markets")

print(f"\n{'='*80}")
print(f"✓ TOTAL MARKETS FETCHED: {len(all_companies_markets)}")
print(f"{'='*80}")

companies_markets_raw = all_companies_markets


FETCHING MARKETS FOR COMPANIES SERIES

Fetching settled markets for 349 series...
(This may take a few minutes...)

  [1/349] KXDANGROSSMETA: 1 markets
  [2/349] KXTESLAPROD: 31 markets
  Progress: 5/349 series processed, 32 total markets
  [6/349] KXANTITRUSTOAIMSFT: 1 markets
  Rate limited, waiting 2 seconds...
  [7/349] KXGROKMACOS: 1 markets
  [8/349] KXNEWPRODUCTFALLOUT3REM: 1 markets
  [9/349] KXACQANNOUNEWARNER: 5 markets
  [10/349] KXTESLAYL: 1 markets
  Progress: 10/349 series processed, 41 total markets
  [11/349] KXWEWORKCEO: 1 markets
  Rate limited, waiting 2 seconds...
  Progress: 15/349 series processed, 42 total markets
  [16/349] KXCRYPTOPAY: 1 markets
  [17/349] KXAAPLCEOCHANGE: 1 markets
  Rate limited, waiting 2 seconds...
  [18/349] KXROGANBOARD: 1 markets
  [19/349] KXAPPLEFOLD: 1 markets
  Progress: 20/349 series processed, 46 total markets
  [22/349] KXJOINMETA: 8 markets
  Rate limited, waiting 2 seconds...
  Progress: 25/349 series processed, 54 total market

In [9]:
# ============================================================================
# STEP 3: Filter by Duration (1 Week to 1 Month) and Liquidity
# ============================================================================

print("\n" + "="*80)
print("APPLYING FILTERS: DURATION 1 WEEK - 1 MONTH, LIQUIDITY >= $100,000")
print("="*80)

MIN_DURATION_HOURS_COMP = 24
MAX_DURATION_HOURS_COMP = 30 * 24
MIN_LIQUIDITY_DOLLARS_COMP = 100

filtered_companies_markets = []
filter_stats_comp = {
    'total': len(companies_markets_raw),
    'duration_pass': 0,
    'liquidity_pass': 0,
    'both_pass': 0,
    'too_short': 0,
    'too_long': 0,
    'liquidity_fail': 0
}

for market in companies_markets_raw:
    try:
        open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
        close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
        duration_hours = (close_time - open_time).total_seconds() / 3600
        liquidity_dollars = float(market.get('liquidity_dollars', '0'))
        
        duration_ok = MIN_DURATION_HOURS_COMP <= duration_hours <= MAX_DURATION_HOURS_COMP
        liquidity_ok = liquidity_dollars >= MIN_LIQUIDITY_DOLLARS_COMP
        
        if duration_ok:
            filter_stats_comp['duration_pass'] += 1
        else:
            if duration_hours < MIN_DURATION_HOURS_COMP:
                filter_stats_comp['too_short'] += 1
            else:
                filter_stats_comp['too_long'] += 1
        
        if liquidity_ok:
            filter_stats_comp['liquidity_pass'] += 1
        else:
            filter_stats_comp['liquidity_fail'] += 1
        
        if duration_ok and liquidity_ok:
            market['duration_hours'] = duration_hours
            market['liquidity_dollars_float'] = liquidity_dollars
            filtered_companies_markets.append(market)
            filter_stats_comp['both_pass'] += 1
    except Exception as e:
        continue

print(f"\nFilter Results:")
print(f"  Total markets:                {filter_stats_comp['total']}")
print(f"  Duration filter (1w - 1m):    {filter_stats_comp['duration_pass']} passed")
print(f"    - Too short (< 1 week):     {filter_stats_comp['too_short']}")
print(f"    - Too long (> 1 month):     {filter_stats_comp['too_long']}")
print(f"  Liquidity filter (>=$100k):   {filter_stats_comp['liquidity_pass']} passed")
print(f"  ✓ Both filters passed:        {filter_stats_comp['both_pass']} markets")

if filter_stats_comp['both_pass'] == 0:
    print("\n" + "⚠"*40)
    print("⚠ NO MARKETS PASSED - LOWERING LIQUIDITY THRESHOLD")
    print("⚠"*40)
    
    for min_liq in [50000, 25000, 10000, 5000, 0]:
        filtered_companies_markets = []
        for market in companies_markets_raw:
            try:
                open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
                close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
                duration_hours = (close_time - open_time).total_seconds() / 3600
                liquidity_dollars = float(market.get('liquidity_dollars', '0'))
                
                if MIN_DURATION_HOURS_COMP <= duration_hours <= MAX_DURATION_HOURS_COMP and liquidity_dollars >= min_liq:
                    market['duration_hours'] = duration_hours
                    market['liquidity_dollars_float'] = liquidity_dollars
                    filtered_companies_markets.append(market)
            except:
                continue
        
        if len(filtered_companies_markets) > 0:
            print(f"\n✓ Found {len(filtered_companies_markets)} markets with liquidity >= ${min_liq:,}")
            MIN_LIQUIDITY_DOLLARS_COMP = min_liq
            break

print(f"\n{'='*80}")
print(f"✓ QUALIFIED MARKETS: {len(filtered_companies_markets)}")
print(f"{'='*80}")

if filtered_companies_markets:
    print("\nSample qualified markets (first 5):")
    for i, m in enumerate(filtered_companies_markets[:5], 1):
        print(f"\n  {i}. {m['ticker']}")
        print(f"     Title: {m.get('title', 'N/A')}")
        print(f"     Duration: {m['duration_hours']/24:.2f} days")
        print(f"     Liquidity: ${m['liquidity_dollars_float']:,.2f}")
        print(f"     Result: {m.get('result', 'N/A')}")

companies_markets_qualified = filtered_companies_markets


APPLYING FILTERS: DURATION 1 WEEK - 1 MONTH, LIQUIDITY >= $100,000

Filter Results:
  Total markets:                3255
  Duration filter (1w - 1m):    2668 passed
    - Too short (< 1 week):     62
    - Too long (> 1 month):     525
  Liquidity filter (>=$100k):   121 passed
  ✓ Both filters passed:        32 markets

✓ QUALIFIED MARKETS: 32

Sample qualified markets (first 5):

  1. KXDANGROSSMETA-25SEP
     Title: Will Daniel Gross join Meta before Sep 2025?
     Duration: 9.73 days
     Liquidity: $655.70
     Result: yes

  2. KXMERCHLISTPALANTIR-25JUL-PANTS
     Title: What kind of new merch will Palantir drop this year?
     Duration: 17.72 days
     Liquidity: $175.13
     Result: no

  3. KXISMPMI-25AUG-52
     Title: ISM Manufacturing PMI in Aug 2025?
     Duration: 17.00 days
     Liquidity: $1,142.88
     Result: no

  4. KXISMPMI-25AUG-51
     Title: ISM Manufacturing PMI in Aug 2025?
     Duration: 17.00 days
     Liquidity: $1,695.42
     Result: no

  5. KXISMPMI-25A

In [ ]:
# ============================================================================
# STEP 4: Group by Event & Calculate Entry Points (90% Time Elapsed)
# ============================================================================

print("\n" + "="*80)
print("GROUPING MARKETS BY EVENT & CALCULATING ENTRY POINTS (90% TIME ELAPSED)")
print("="*80)

events_dict_comp = {}

for market in companies_markets_qualified:
    event_ticker = market.get('event_ticker', market['ticker'])
    
    if event_ticker not in events_dict_comp:
        events_dict_comp[event_ticker] = []
    
    # Calculate entry at 90% of market duration elapsed
    open_time = datetime.fromisoformat(market['open_time'].replace('Z', '+00:00'))
    close_time = datetime.fromisoformat(market['close_time'].replace('Z', '+00:00'))
    market_duration = close_time - open_time
    entry_time = open_time + (market_duration * 0.9)  # 90% elapsed
    
    market['open_time_dt'] = open_time
    market['entry_time'] = entry_time
    market['close_time_dt'] = close_time
    market['time_to_expiry_hours'] = (close_time - entry_time).total_seconds() / 3600
    
    events_dict_comp[event_ticker].append(market)

print(f"\n✓ Found {len(events_dict_comp)} unique events")

# Show sample entry timing
if companies_markets_qualified:
    sample = companies_markets_qualified[0]
    duration_days = sample['duration_hours'] / 24
    time_to_expiry = sample['time_to_expiry_hours']
    print(f"\nSample entry timing:")
    print(f"  Market duration: {duration_days:.2f} days")
    print(f"  Entry point: {duration_days * 0.9:.2f} days after open")
    print(f"  Time remaining at entry: {time_to_expiry:.2f} hours ({time_to_expiry/24:.2f} days)")

event_classification_comp = {'single': [], 'multi': []}
for event_ticker, markets in events_dict_comp.items():
    if len(markets) == 1:
        event_classification_comp['single'].append(event_ticker)
    else:
        event_classification_comp['multi'].append(event_ticker)

print(f"\nEvent Classification:")
print(f"  Single-outcome events: {len(event_classification_comp['single'])}")
print(f"  Multi-outcome events:  {len(event_classification_comp['multi'])}")

multi_decision_events_comp = {k: v for k, v in events_dict_comp.items() if len(v) >= 2}
print(f"\n✓ Multi-decision events: {len(multi_decision_events_comp)}")

if multi_decision_events_comp:
    print("\nSample multi-decision events (first 3):")
    for i, (event_ticker, markets) in enumerate(list(multi_decision_events_comp.items())[:3], 1):
        print(f"\n  {i}. Event: {event_ticker} ({len(markets)} markets)")
        for m in markets[:2]:
            print(f"     - {m['ticker']}: ${m.get('yes_bid', 0)/100:.2f} (result: {m.get('result', 'N/A')})")

all_tradeable_events_comp = events_dict_comp.copy()
print(f"\n✓ Total tradeable events: {len(all_tradeable_events_comp)}")

# ============================================================================
# STEP 5: Organize Events by Entry Date
# ============================================================================

print("\n" + "="*80)
print("ORGANIZING EVENTS BY ENTRY DATE")
print("="*80)

events_by_entry_date_comp = {}

for event_ticker, markets in all_tradeable_events_comp.items():
    entry_time = markets[0]['entry_time']
    entry_date = entry_time.date()
    
    if entry_date not in events_by_entry_date_comp:
        events_by_entry_date_comp[entry_date] = []
    
    events_by_entry_date_comp[entry_date].append({
        'event_ticker': event_ticker,
        'markets': markets,
        'entry_time': entry_time,
        'num_markets': len(markets)
    })

sorted_dates_comp = sorted(events_by_entry_date_comp.keys())

print(f"\n✓ Trading spans {len(sorted_dates_comp)} unique dates")
if sorted_dates_comp:
    print(f"  First entry date: {sorted_dates_comp[0]}")
    print(f"  Last entry date:  {sorted_dates_comp[-1]}")

events_per_day_comp = [len(events) for events in events_by_entry_date_comp.values()]
if events_per_day_comp:
    print(f"\nEvents per day:")
    print(f"  Average: {np.mean(events_per_day_comp):.2f}")
    print(f"  Max: {np.max(events_per_day_comp)}")

# ============================================================================
# STEP 6: Simulate Trading Strategy (10% Liquidity Position Sizing)
# ============================================================================

print("\n" + "="*80)
print("SIMULATING TRADING STRATEGY (10% LIQUIDITY POSITION SIZING)")
print("="*80)

INITIAL_CAPITAL_COMP = 100000
LIQUIDITY_PERCENTAGE = 0.10  # 10% of market liquidity
YES_ALLOCATION_COMP = 0.70   # 70% of position on YES
NO_ALLOCATION_COMP = 0.30    # 30% of position split across NOs

all_trades_comp = []
trade_id_comp = 0
total_exposure_by_date_comp = {}

print(f"Position sizing: 10% of each market's liquidity")
print(f"Multi-decision split: 70% YES favorite, 30% across NOs\n")

for date in sorted_dates_comp:
    events_today = events_by_entry_date_comp[date]
    if len(events_today) == 0:
        continue
    
    daily_exposure = 0
    
    for event in events_today:
        markets = event['markets']
        
        if len(markets) >= 2:
            # Multi-decision: YES on favorite, NO on others
            favorite = max(markets, key=lambda m: m.get('yes_bid', 0))
            others = [m for m in markets if m != favorite]
            
            # Position size = 10% of favorite's liquidity
            base_position = favorite['liquidity_dollars_float'] * LIQUIDITY_PERCENTAGE
            yes_size = base_position * YES_ALLOCATION_COMP
            no_size = base_position * NO_ALLOCATION_COMP
            no_per_market = no_size / len(others) if others else 0
            
            # YES on favorite
            yes_price = favorite.get('yes_ask', favorite.get('yes_bid', 50))
            yes_contracts = (yes_size * 100) / yes_price if yes_price > 0 else 0
            yes_pnl = yes_contracts * (100 - yes_price) / 100 if favorite.get('result') == 'yes' else (-yes_size if favorite.get('result') == 'no' else 0)
            
            trade_id_comp += 1
            all_trades_comp.append({
                'trade_id': trade_id_comp, 'event_ticker': event['event_ticker'], 'ticker': favorite['ticker'],
                'entry_date': date, 'close_time': favorite['close_time_dt'], 'side': 'YES',
                'price': yes_price / 100, 'investment': yes_size, 'result': favorite.get('result', 'unknown'),
                'pnl': yes_pnl, 'liquidity': favorite.get('liquidity_dollars_float', 0),
                'duration_days': favorite.get('duration_hours', 0) / 24,
                'time_to_expiry_days': favorite['time_to_expiry_hours'] / 24,
                'entry_time': favorite['entry_time']
            })
            daily_exposure += yes_size
            
            # NO on others
            for other in others:
                no_price = other.get('no_ask', other.get('no_bid', 50))
                no_contracts = (no_per_market * 100) / no_price if no_price > 0 else 0
                no_pnl = no_contracts * (100 - no_price) / 100 if other.get('result') == 'no' else (-no_per_market if other.get('result') == 'yes' else 0)
                
                trade_id_comp += 1
                all_trades_comp.append({
                    'trade_id': trade_id_comp, 'event_ticker': event['event_ticker'], 'ticker': other['ticker'],
                    'entry_date': date, 'close_time': other['close_time_dt'], 'side': 'NO',
                    'price': no_price / 100, 'investment': no_per_market, 'result': other.get('result', 'unknown'),
                    'pnl': no_pnl, 'liquidity': other.get('liquidity_dollars_float', 0),
                    'duration_days': other.get('duration_hours', 0) / 24,
                    'time_to_expiry_days': other['time_to_expiry_hours'] / 24,
                    'entry_time': other['entry_time']
                })
                daily_exposure += no_per_market
        else:
            # Single binary market
            market = markets[0]
            position_size = market['liquidity_dollars_float'] * LIQUIDITY_PERCENTAGE
            yes_bid = market.get('yes_bid', 50)
            
            if yes_bid >= 50:
                # Buy YES
                price = market.get('yes_ask', yes_bid)
                contracts = (position_size * 100) / price if price > 0 else 0
                pnl = contracts * (100 - price) / 100 if market.get('result') == 'yes' else (-position_size if market.get('result') == 'no' else 0)
                side = 'YES'
            else:
                # Buy NO
                price = market.get('no_ask', 100 - yes_bid)
                contracts = (position_size * 100) / price if price > 0 else 0
                pnl = contracts * (100 - price) / 100 if market.get('result') == 'no' else (-position_size if market.get('result') == 'yes' else 0)
                side = 'NO'
            
            trade_id_comp += 1
            all_trades_comp.append({
                'trade_id': trade_id_comp, 'event_ticker': event['event_ticker'], 'ticker': market['ticker'],
                'entry_date': date, 'close_time': market['close_time_dt'], 'side': side,
                'price': price / 100, 'investment': position_size, 'result': market.get('result', 'unknown'),
                'pnl': pnl, 'liquidity': market.get('liquidity_dollars_float', 0),
                'duration_days': market.get('duration_hours', 0) / 24,
                'time_to_expiry_days': market['time_to_expiry_hours'] / 24,
                'entry_time': market['entry_time']
            })
            daily_exposure += position_size
    
    total_exposure_by_date_comp[date] = daily_exposure

print(f"✓ Simulation complete: {len(all_trades_comp)} trades")

# Calculate exposure statistics
if total_exposure_by_date_comp:
    exposures = list(total_exposure_by_date_comp.values())
    total_capital_deployed = sum(t['investment'] for t in all_trades_comp)
    
    print(f"\nExposure Analysis:")
    print(f"  Total capital deployed: ${total_capital_deployed:,.2f}")
    print(f"  Avg daily exposure: ${np.mean(exposures):,.2f}")
    print(f"  Max daily exposure: ${np.max(exposures):,.2f}")
    print(f"  Min daily exposure: ${np.min(exposures):,.2f}")

trades_df_comp = pd.DataFrame(all_trades_comp)

if not trades_df_comp.empty:
    print(f"\nSample trades (first 5):")
    for i, row in trades_df_comp.head(5).iterrows():
        print(f"  {row['ticker'][:30]:<30} {row['side']:<4} ${row['investment']:>8,.0f} (liq: ${row['liquidity']:>8,.0f}) TTL: {row['time_to_expiry_days']:.1f}d")

print(f"{'='*80}")

In [ ]:
# ============================================================================
# STEP 5: Organize Events by Entry Date
# ============================================================================

print("\n" + "="*80)
print("ORGANIZING EVENTS BY ENTRY DATE")
print("="*80)

events_by_entry_date_comp = {}

for event_ticker, markets in all_tradeable_events_comp.items():
    entry_time = markets[0]['entry_time']
    entry_date = entry_time.date()
    
    if entry_date not in events_by_entry_date_comp:
        events_by_entry_date_comp[entry_date] = []
    
    events_by_entry_date_comp[entry_date].append({
        'event_ticker': event_ticker,
        'markets': markets,
        'entry_time': entry_time,
        'num_markets': len(markets)
    })

sorted_dates_comp = sorted(events_by_entry_date_comp.keys())

print(f"\n✓ Trading spans {len(sorted_dates_comp)} unique dates")
if sorted_dates_comp:
    print(f"  First entry date: {sorted_dates_comp[0]}")
    print(f"  Last entry date:  {sorted_dates_comp[-1]}")

events_per_day_comp = [len(events) for events in events_by_entry_date_comp.values()]
if events_per_day_comp:
    print(f"\nEvents per day:")
    print(f"  Average: {np.mean(events_per_day_comp):.2f}")
    print(f"  Max: {np.max(events_per_day_comp)}")

print(f"\n{'='*80}")

In [ ]:
# ============================================================================
# STEP 6: Simulate Trading Strategy
# ============================================================================

print("\n" + "="*80)
print("SIMULATING TRADING STRATEGY")
print("="*80)

INITIAL_CAPITAL_COMP = 100000
DAILY_BUDGET_COMP = 10000
YES_ALLOCATION_COMP = 0.70
NO_ALLOCATION_COMP = 0.30

all_trades_comp = []
trade_id_comp = 0

print(f"\nStrategy: $100k capital, $10k daily, 70% YES / 30% NO split")
print(f"\nProcessing {len(sorted_dates_comp)} trading days...")

for date in sorted_dates_comp:
    events_today = events_by_entry_date_comp[date]
    if len(events_today) == 0:
        continue
    
    budget_per_event = DAILY_BUDGET_COMP / len(events_today)
    
    for event in events_today:
        markets = event['markets']
        
        if len(markets) >= 2:
            # Multi-decision
            favorite = max(markets, key=lambda m: m.get('yes_bid', 0))
            others = [m for m in markets if m != favorite]
            
            yes_size = budget_per_event * YES_ALLOCATION_COMP
            no_size = budget_per_event * NO_ALLOCATION_COMP
            no_per_market = no_size / len(others) if others else 0
            
            # YES on favorite
            yes_price = favorite.get('yes_ask', favorite.get('yes_bid', 50))
            yes_contracts = (yes_size * 100) / yes_price if yes_price > 0 else 0
            yes_pnl = yes_contracts * (100 - yes_price) / 100 if favorite.get('result') == 'yes' else (-yes_size if favorite.get('result') == 'no' else 0)
            
            trade_id_comp += 1
            all_trades_comp.append({
                'trade_id': trade_id_comp, 'event_ticker': event['event_ticker'], 'ticker': favorite['ticker'],
                'entry_date': date, 'close_time': favorite['close_time_dt'], 'side': 'YES',
                'price': yes_price / 100, 'investment': yes_size, 'result': favorite.get('result', 'unknown'),
                'pnl': yes_pnl, 'liquidity': favorite.get('liquidity_dollars_float', 0),
                'duration_days': favorite.get('duration_hours', 0) / 24
            })
            
            # NO on others
            for other in others:
                no_price = other.get('no_ask', other.get('no_bid', 50))
                no_contracts = (no_per_market * 100) / no_price if no_price > 0 else 0
                no_pnl = no_contracts * (100 - no_price) / 100 if other.get('result') == 'no' else (-no_per_market if other.get('result') == 'yes' else 0)
                
                trade_id_comp += 1
                all_trades_comp.append({
                    'trade_id': trade_id_comp, 'event_ticker': event['event_ticker'], 'ticker': other['ticker'],
                    'entry_date': date, 'close_time': other['close_time_dt'], 'side': 'NO',
                    'price': no_price / 100, 'investment': no_per_market, 'result': other.get('result', 'unknown'),
                    'pnl': no_pnl, 'liquidity': other.get('liquidity_dollars_float', 0),
                    'duration_days': other.get('duration_hours', 0) / 24
                })
        else:
            # Single binary
            market = markets[0]
            yes_bid = market.get('yes_bid', 50)
            
            if yes_bid >= 50:
                price = market.get('yes_ask', yes_bid)
                contracts = (budget_per_event * 100) / price if price > 0 else 0
                pnl = contracts * (100 - price) / 100 if market.get('result') == 'yes' else (-budget_per_event if market.get('result') == 'no' else 0)
                side = 'YES'
            else:
                price = market.get('no_ask', 100 - yes_bid)
                contracts = (budget_per_event * 100) / price if price > 0 else 0
                pnl = contracts * (100 - price) / 100 if market.get('result') == 'no' else (-budget_per_event if market.get('result') == 'yes' else 0)
                side = 'NO'
            
            trade_id_comp += 1
            all_trades_comp.append({
                'trade_id': trade_id_comp, 'event_ticker': event['event_ticker'], 'ticker': market['ticker'],
                'entry_date': date, 'close_time': market['close_time_dt'], 'side': side,
                'price': price / 100, 'investment': budget_per_event, 'result': market.get('result', 'unknown'),
                'pnl': pnl, 'liquidity': market.get('liquidity_dollars_float', 0),
                'duration_days': market.get('duration_hours', 0) / 24
            })

print(f"\n✓ Simulation complete: {len(all_trades_comp)} trades")
print(f"{'='*80}")

trades_df_comp = pd.DataFrame(all_trades_comp)

In [ ]:
# ============================================================================
# STEP 7: Build Equity Curve, Calculate Metrics, Analyze Top 5%, and Summary
# ============================================================================

print("\n" + "="*80)
print("PERFORMANCE ANALYSIS")
print("="*80)

if trades_df_comp.empty:
    print("\n⚠ WARNING: No trades to analyze!")
else:
    # ========== BUILD EQUITY CURVE ==========
    trades_df_comp['close_date'] = trades_df_comp['close_time'].dt.date
    daily_pnl_comp = trades_df_comp.groupby('close_date')['pnl'].sum().sort_index()
    
    equity_curve_comp = pd.Series(dtype=float)
    equity_curve_comp[daily_pnl_comp.index[0]] = INITIAL_CAPITAL_COMP
    
    for date in daily_pnl_comp.index:
        prev_equity = equity_curve_comp.iloc[-1]
        equity_curve_comp[date] = prev_equity + daily_pnl_comp[date]
    
    equity_df_comp = pd.DataFrame({
        'date': equity_curve_comp.index,
        'equity': equity_curve_comp.values,
        'daily_pnl': [0] + list(daily_pnl_comp.values)
    })
    
    equity_df_comp['daily_return'] = equity_df_comp['equity'].pct_change()
    equity_df_comp['cumulative_return'] = (equity_df_comp['equity'] / INITIAL_CAPITAL_COMP - 1) * 100
    equity_df_comp['running_max'] = equity_df_comp['equity'].cummax()
    equity_df_comp['drawdown'] = (equity_df_comp['equity'] - equity_df_comp['running_max']) / equity_df_comp['running_max'] * 100
    
    # ========== CALCULATE METRICS ==========
    total_return_pct_comp = (equity_df_comp['equity'].iloc[-1] / INITIAL_CAPITAL_COMP - 1) * 100
    total_pnl_comp = equity_df_comp['equity'].iloc[-1] - INITIAL_CAPITAL_COMP
    
    days_trading_comp = (equity_df_comp['date'].iloc[-1] - equity_df_comp['date'].iloc[0]).days
    years_trading_comp = days_trading_comp / 365.25
    cagr_comp = ((equity_df_comp['equity'].iloc[-1] / INITIAL_CAPITAL_COMP) ** (1 / years_trading_comp) - 1) * 100 if years_trading_comp > 0 else 0
    
    daily_returns_comp = equity_df_comp['daily_return'].dropna()
    if len(daily_returns_comp) > 1:
        sharpe_ratio_comp = (daily_returns_comp.mean() / daily_returns_comp.std() * np.sqrt(252)) if daily_returns_comp.std() > 0 else 0
        downside_std_comp = daily_returns_comp[daily_returns_comp < 0].std() if len(daily_returns_comp[daily_returns_comp < 0]) > 1 else 0
        sortino_ratio_comp = (daily_returns_comp.mean() / downside_std_comp * np.sqrt(252)) if downside_std_comp > 0 else 0
    else:
        sharpe_ratio_comp = 0
        sortino_ratio_comp = 0
    
    max_drawdown_comp = equity_df_comp['drawdown'].min()
    avg_drawdown_comp = equity_df_comp['drawdown'][equity_df_comp['drawdown'] < 0].mean()
    
    total_trades_comp = len(trades_df_comp)
    winning_trades_comp = len(trades_df_comp[trades_df_comp['pnl'] > 0])
    losing_trades_comp = len(trades_df_comp[trades_df_comp['pnl'] < 0])
    win_rate_comp = (winning_trades_comp / total_trades_comp * 100) if total_trades_comp > 0 else 0
    
    avg_win_comp = trades_df_comp[trades_df_comp['pnl'] > 0]['pnl'].mean() if winning_trades_comp > 0 else 0
    avg_loss_comp = trades_df_comp[trades_df_comp['pnl'] < 0]['pnl'].mean() if losing_trades_comp > 0 else 0
    
    profit_factor_comp = (trades_df_comp[trades_df_comp['pnl'] > 0]['pnl'].sum() / 
                         abs(trades_df_comp[trades_df_comp['pnl'] < 0]['pnl'].sum())) if losing_trades_comp > 0 else float('inf')
    expectancy_comp = trades_df_comp['pnl'].mean()
    
    # ========== TOP 5% LIQUIDITY ==========
    liquidity_95th_percentile_comp = trades_df_comp['liquidity'].quantile(0.95)
    top_5_pct_trades_comp = trades_df_comp[trades_df_comp['liquidity'] >= liquidity_95th_percentile_comp]
    
    if len(top_5_pct_trades_comp) > 0:
        top_5_pnl_comp = top_5_pct_trades_comp['pnl'].sum()
        top_5_win_rate_comp = (len(top_5_pct_trades_comp[top_5_pct_trades_comp['pnl'] > 0]) / len(top_5_pct_trades_comp) * 100)
        
        top_5_daily_pnl_comp = top_5_pct_trades_comp.groupby('close_date')['pnl'].sum().sort_index()
        top_5_equity_comp = pd.Series(dtype=float)
        top_5_equity_comp[top_5_daily_pnl_comp.index[0]] = INITIAL_CAPITAL_COMP
        for date in top_5_daily_pnl_comp.index:
            top_5_equity_comp[date] = top_5_equity_comp.iloc[-1] + top_5_daily_pnl_comp[date]
        
        top_5_returns_comp = top_5_equity_comp.pct_change().dropna()
        top_5_sharpe_comp = (top_5_returns_comp.mean() / top_5_returns_comp.std() * np.sqrt(252)) if len(top_5_returns_comp) > 1 and top_5_returns_comp.std() > 0 else 0
        top_5_total_return_comp = (top_5_equity_comp.iloc[-1] / INITIAL_CAPITAL_COMP - 1) * 100
    else:
        top_5_sharpe_comp = 0
        top_5_win_rate_comp = 0
        top_5_total_return_comp = 0
    
    # ========== DISPLAY RESULTS ==========
    print("\n" + "="*80)
    print("COMPANIES MARKETS - STRATEGY PERFORMANCE")
    print("="*80)
    
    print(f"\n📊 RETURNS:")
    print(f"  Total Return:        {total_return_pct_comp:>10.2f}%")
    print(f"  Total P&L:           ${total_pnl_comp:>10,.2f}")
    print(f"  CAGR:                {cagr_comp:>10.2f}%")
    print(f"  Trading Period:      {days_trading_comp} days ({years_trading_comp:.2f} years)")
    
    print(f"\n📉 RISK METRICS:")
    print(f"  Sharpe Ratio:        {sharpe_ratio_comp:>10.2f}")
    print(f"  Sortino Ratio:       {sortino_ratio_comp:>10.2f}")
    print(f"  Max Drawdown:        {max_drawdown_comp:>10.2f}%")
    print(f"  Avg Drawdown:        {avg_drawdown_comp:>10.2f}%")
    
    print(f"\n💰 TRADE STATISTICS:")
    print(f"  Total Trades:        {total_trades_comp:>10,}")
    print(f"  Winning Trades:      {winning_trades_comp:>10,}")
    print(f"  Win Rate:            {win_rate_comp:>10.1f}%")
    print(f"  Avg Win:             ${avg_win_comp:>10,.2f}")
    print(f"  Avg Loss:            ${avg_loss_comp:>10,.2f}")
    print(f"  Profit Factor:       {profit_factor_comp if profit_factor_comp != float('inf') else 'N/A':>10}")
    print(f"  Expectancy/Trade:    ${expectancy_comp:>10,.2f}")
    
    print(f"\n🔝 TOP 5% LIQUIDITY (>= ${liquidity_95th_percentile_comp:,.2f}):")
    print(f"  Trades:              {len(top_5_pct_trades_comp):>10,}")
    print(f"  Total Return:        {top_5_total_return_comp:>10.2f}%")
    print(f"  Sharpe Ratio:        {top_5_sharpe_comp:>10.2f}")
    print(f"  Win Rate:            {top_5_win_rate_comp:>10.1f}%")
    
    print(f"\n📊 MONTHLY BREAKDOWN:")
    trades_df_comp['month'] = pd.to_datetime(trades_df_comp['close_date']).dt.to_period('M')
    monthly_pnl = trades_df_comp.groupby('month')['pnl'].sum()
    monthly_trades = trades_df_comp.groupby('month').size()
    
    print(f"  {'Month':<12} {'Trades':>8} {'P&L':>12}")
    print(f"  {'-'*35}")
    for month in monthly_pnl.index:
        print(f"  {str(month):<12} {monthly_trades[month]:>8,} ${monthly_pnl[month]:>11,.2f}")
    
    print(f"\n💡 KEY INSIGHTS:")
    if total_return_pct_comp > 0:
        print(f"  ✓ Strategy profitable: {total_return_pct_comp:.2f}% return")
    else:
        print(f"  ✗ Strategy unprofitable: {total_return_pct_comp:.2f}% return")
    
    if sharpe_ratio_comp > 1:
        print(f"  ✓ Excellent risk-adjusted returns (Sharpe {sharpe_ratio_comp:.2f})")
    elif sharpe_ratio_comp > 0.5:
        print(f"  ~ Moderate risk-adjusted returns (Sharpe {sharpe_ratio_comp:.2f})")
    else:
        print(f"  ✗ Poor risk-adjusted returns (Sharpe {sharpe_ratio_comp:.2f})")
    
    if win_rate_comp > 55:
        print(f"  ✓ High win rate ({win_rate_comp:.1f}%)")
    elif win_rate_comp > 45:
        print(f"  ~ Moderate win rate ({win_rate_comp:.1f}%)")
    else:
        print(f"  ✗ Low win rate ({win_rate_comp:.1f}%)")
    
    if abs(max_drawdown_comp) < 10:
        print(f"  ✓ Low max drawdown ({abs(max_drawdown_comp):.1f}%)")
    elif abs(max_drawdown_comp) < 20:
        print(f"  ~ Moderate max drawdown ({abs(max_drawdown_comp):.1f}%)")
    else:
        print(f"  ✗ High max drawdown ({abs(max_drawdown_comp):.1f}%)")
    
    print(f"\n💾 DATA EXPORTS:")
    print(f"  trades_df_comp          - All {len(trades_df_comp)} trades")
    print(f"  equity_df_comp          - Daily equity curve")
    print(f"  top_5_pct_trades_comp   - Top 5% liquidity trades")

print(f"\n{'='*80}")
print("BACKTEST COMPLETE!")
print("="*80)